In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='abcbank_secretScope')]

In [0]:
dbutils.secrets.list("abcbank_secretScope")

[SecretMetadata(key='client-id'),
 SecretMetadata(key='client-secret'),
 SecretMetadata(key='directory-id')]

In [0]:
client_id = dbutils.secrets.get("abcbank_secretScope", "client-id")
client_secret = dbutils.secrets.get("abcbank_secretScope", "client-secret")
directory_id = dbutils.secrets.get("abcbank_secretScope", "directory-id")

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{directory_id}/oauth2/token"}

In [0]:
storage_account = "dlsaabcbank"

container_raw = "bronze"
container_processed = "silver"

source_raw = f"abfss://{container_raw}@{storage_account}.dfs.core.windows.net/"
mount_point_raw = f"/mnt/{storage_account}/{container_raw}"

source_processed = f"abfss://{container_processed}@{storage_account}.dfs.core.windows.net/"
mount_point_processed = f"/mnt/{storage_account}/{container_processed}"

In [0]:
if any(mount.mountPoint == mount_point_raw for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(mount_point_raw)
    print(f"Unmount existing mount at {mount_point_raw}")

try:
    dbutils.fs.mount(
        source = source_raw,
        mount_point = mount_point_raw,
        extra_configs = configs
    )
    print(f"Mounted successfully at {mount_point_raw}")
except Exception as e:
    print(f"Error mounting: {mount_point_raw}")

/mnt/dlsaabcbank/bronze has been unmounted.
Unmount existing mount at /mnt/dlsaabcbank/bronze
Mounted successfully at /mnt/dlsaabcbank/bronze


In [0]:
if any(mount.mountPoint == mount_point_processed for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(mount_point_processed)
    print(f"Unmount existing mount at {mount_point_processed}")

try:
    dbutils.fs.mount(
        source = source_processed,
        mount_point = mount_point_processed,
        extra_configs = configs
    )
    print(f"Mounted successfully at {mount_point_processed}")
except Exception as e:
    print(f"Error mounting: {mount_point_processed}")

/mnt/dlsaabcbank/silver has been unmounted.
Unmount existing mount at /mnt/dlsaabcbank/silver
Mounted successfully at /mnt/dlsaabcbank/silver


In [0]:
dbutils.fs.ls(f"{mount_point_raw}")

[FileInfo(path='dbfs:/mnt/dlsaabcbank/bronze/accounts_2024-11-07T08:21:28.1125871Z', name='accounts_2024-11-07T08:21:28.1125871Z', size=1354, modificationTime=1730967703000),
 FileInfo(path='dbfs:/mnt/dlsaabcbank/bronze/accounts_2024-11-07T08:24:49.8708807Z', name='accounts_2024-11-07T08:24:49.8708807Z', size=1352, modificationTime=1730967909000),
 FileInfo(path='dbfs:/mnt/dlsaabcbank/bronze/accounts_2024-11-07T08:27:35.3790980Z', name='accounts_2024-11-07T08:27:35.3790980Z', size=1356, modificationTime=1730968069000),
 FileInfo(path='dbfs:/mnt/dlsaabcbank/bronze/customers_2024-11-07T08:21:19.7005557Z', name='customers_2024-11-07T08:21:19.7005557Z', size=2925, modificationTime=1730967694000),
 FileInfo(path='dbfs:/mnt/dlsaabcbank/bronze/customers_2024-11-07T08:24:41.7249519Z', name='customers_2024-11-07T08:24:41.7249519Z', size=2828, modificationTime=1730967896000),
 FileInfo(path='dbfs:/mnt/dlsaabcbank/bronze/customers_2024-11-07T08:27:30.5506264Z', name='customers_2024-11-07T08:27:30

In [0]:
customer_files = [file.path for file in dbutils.fs.ls(mount_point_raw) if "customers_" in file.name]
account_files = [file.path for file in dbutils.fs.ls(mount_point_raw) if "accounts_" in file.name]
transaction_files = [file.path for file in dbutils.fs.ls(mount_point_raw) if "transactions_" in file.name]
loan_files = [file.path for file in dbutils.fs.ls(mount_point_raw) if "loans_" in file.name]
loan_payment_files = [file.path for file in dbutils.fs.ls(mount_point_raw) if "loan_payments_" in file.name]

In [0]:
from pyspark.sql.functions import current_timestamp, when, col

df_customers = spark.read.parquet(*customer_files, header=True, inferSchema=True) \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .dropna(subset=["customer_id", "first_name"]) \
    .dropDuplicates(["customer_id"]) \
    .orderBy("customer_id")
df_customers.show(15)

total_customer_records = df_customers.count()
print(f"Total customer records: {total_customer_records}")

+-----------+-----------+---------+---------------+-------------+-----+------+--------------------+
|customer_id| first_name|last_name|        address|         city|state|   zip| ingestion_timestamp|
+-----------+-----------+---------+---------------+-------------+-----+------+--------------------+
|          1|       John|      Doe|     123 Elm St|      Toronto|   ON|M4B1B3|2024-11-07 08:29:...|
|         10|        Ava| Anderson|909 Cypress Ave|  Quebec City|   QC|G1A0A1|2024-11-07 08:29:...|
|         11|  Alexander|   Thomas| 1010 Willow Rd|   St. John's|   NL|A1A0A1|2024-11-07 08:29:...|
|         12|   Isabella|      Lee| 1111 Poplar St|  Fredericton|   NB|E3B0A1|2024-11-07 08:29:...|
|         13|     Daniel|   Harris|  1212 Ash Blvd|Charlottetown|   PE|C1A0A1|2024-11-07 08:29:...|
|         14|     Sophia|    Young|  1313 Beech Dr|  Yellowknife|   NT|X1A0A1|2024-11-07 08:29:...|
|         15|    Matthew|     King|  1414 Cedar Ln|   Whitehorse|   YT|Y1A0A1|2024-11-07 08:29:...|


In [0]:
df_customers.columns

['customer_id',
 'first_name',
 'last_name',
 'address',
 'city',
 'state',
 'zip',
 'ingestion_timestamp']

In [0]:
df_customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = false)



In [0]:
df_accounts = spark.read.parquet(*account_files, header=True, inferSchema=True) \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .withColumn("balance", when(col("balance").isNull(), 0.00).otherwise(col("balance"))) \
    .dropna(subset=["account_id", "customer_id", "account_type"]) \
    .dropDuplicates(["account_id"]) \
    .join(df_customers, on="customer_id", how="inner") \
    .drop(df_customers.ingestion_timestamp) \
    .select("account_id", "customer_id", "account_type", "balance", "ingestion_timestamp") \
    .orderBy("account_id")
df_accounts.show(15)

print(f"Total account records: {df_accounts.count()}")

+----------+-----------+------------+--------+--------------------+
|account_id|customer_id|account_type| balance| ingestion_timestamp|
+----------+-----------+------------+--------+--------------------+
|         1|         45|     Savings| 1000.50|2024-11-07 08:29:...|
|       100|         50|    Checking|10100.00|2024-11-07 08:29:...|
|        11|          3|     Savings| 1100.75|2024-11-07 08:29:...|
|        12|         81|    Checking| 2700.00|2024-11-07 08:29:...|
|        13|         29|     Savings| 1300.25|2024-11-07 08:29:...|
|        14|         64|    Checking| 3200.50|2024-11-07 08:29:...|
|        15|         47|     Savings|  700.75|2024-11-07 08:29:...|
|        16|         18|    Checking| 1400.00|2024-11-07 08:29:...|
|        18|          5|    Checking| 1600.50|2024-11-07 08:29:...|
|        19|         76|     Savings|  400.75|2024-11-07 08:29:...|
|         2|         12|    Checking| 2500.75|2024-11-07 08:29:...|
|        20|         21|    Checking| 2000.00|20

In [0]:
df_accounts.columns

['account_id', 'customer_id', 'account_type', 'balance', 'ingestion_timestamp']

In [0]:
df_accounts.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- account_type: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = false)



In [0]:
df_transactions = spark.read.parquet(*transaction_files, header=True, inferSchema=True) \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .withColumn("transaction_amount", when(col("transaction_amount").isNull(), 0.00).otherwise(col("transaction_amount"))) \
    .dropna(subset=["transaction_id", "account_id"]) \
    .dropDuplicates(["transaction_id"]) \
    .join(df_accounts, on="account_id", how="inner") \
    .drop(df_accounts.ingestion_timestamp) \
    .select("transaction_id", "account_id", "transaction_date", "transaction_amount", "transaction_type", "ingestion_timestamp") \
    .orderBy("transaction_id")
df_transactions.show(15)

print(f"Total transactional recors: {df_transactions.count()}")

+--------------+----------+----------------+------------------+----------------+--------------------+
|transaction_id|account_id|transaction_date|transaction_amount|transaction_type| ingestion_timestamp|
+--------------+----------+----------------+------------------+----------------+--------------------+
|             1|        45|      2024-01-01|            100.50|         Deposit|2024-11-07 08:29:...|
|            10|        92|      2024-01-10|            375.25|      Withdrawal|2024-11-07 08:29:...|
|           100|        50|      2024-04-09|            375.25|      Withdrawal|2024-11-07 08:29:...|
|            11|         3|      2024-01-11|            100.50|         Deposit|2024-11-07 08:29:...|
|            12|        81|      2024-01-12|            200.75|      Withdrawal|2024-11-07 08:29:...|
|            13|        29|      2024-01-13|            150.00|         Deposit|2024-11-07 08:29:...|
|            14|        64|      2024-01-14|            300.25|      Withdrawal|20

In [0]:
df_transactions.columns

['transaction_id',
 'account_id',
 'transaction_date',
 'transaction_amount',
 'transaction_type',
 'ingestion_timestamp']

In [0]:
df_transactions.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- transaction_amount: string (nullable = true)
 |-- transaction_type: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = false)



In [0]:
df_loans = spark.read.parquet(*loan_files, header=True, inferSchema=True) \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .withColumn("loan_amount", when(col("loan_amount").isNull(), 0.00).otherwise(col("loan_amount"))) \
    .withColumn("interest_rate", when(col("interest_rate").isNull(), 0.0).otherwise(col("interest_rate"))) \
    .withColumn("loan_term", when(col("loan_term").isNull(), 1).otherwise(col("loan_term"))) \
    .dropna(subset=["loan_id", "customer_id"]) \
    .dropDuplicates(["loan_id"]) \
    .join(df_customers, on="customer_id", how="inner") \
    .drop(df_customers.ingestion_timestamp) \
    .select("loan_id", "customer_id", "loan_amount", "interest_rate", "loan_term", "ingestion_timestamp") \
    .orderBy("loan_id")
df_loans.show(15)

print(f"Total loan records: {df_loans.count()}")

+-------+-----------+-----------+-------------+---------+--------------------+
|loan_id|customer_id|loan_amount|interest_rate|loan_term| ingestion_timestamp|
+-------+-----------+-----------+-------------+---------+--------------------+
|      1|         45|   10000.50|          5.5|       36|2024-11-07 08:29:...|
|    100|         50|   37500.00|          3.5|       24|2024-11-07 08:29:...|
|     11|          3|   10000.75|          6.0|       60|2024-11-07 08:29:...|
|     12|         81|   20000.00|          3.5|       24|2024-11-07 08:29:...|
|     13|         29|   15000.25|          5.0|       36|2024-11-07 08:29:...|
|     14|         64|   30000.50|          4.0|       48|2024-11-07 08:29:...|
|     15|         47|   25000.75|          6.5|       60|2024-11-07 08:29:...|
|     16|         18|   17500.00|          3.0|       24|2024-11-07 08:29:...|
|     18|          5|   27500.50|          4.5|       48|2024-11-07 08:29:...|
|     19|         76|   32500.75|          6.0|     

In [0]:
df_loans.columns

['loan_id',
 'customer_id',
 'loan_amount',
 'interest_rate',
 'loan_term',
 'ingestion_timestamp']

In [0]:
df_loans.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- loan_amount: string (nullable = true)
 |-- interest_rate: string (nullable = true)
 |-- loan_term: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = false)



In [0]:
df_loan_payments = spark.read.parquet(*loan_payment_files, header=True, inferSchema=True) \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .withColumn("payment_amount", when(col("payment_amount").isNull(), 0.00).otherwise(col("payment_amount"))) \
    .dropna(subset=["payment_id", "loan_id"]) \
    .dropDuplicates(["payment_id"]) \
    .join(df_loans, on="loan_id", how="inner") \
    .drop(df_loans.ingestion_timestamp) \
    .select("payment_id", "loan_id", "payment_date", "payment_amount", "ingestion_timestamp") \
    .orderBy("payment_id")
df_loan_payments.show(15)

print(f"Total loan payment records: {df_loan_payments.count()}")

+----------+-------+------------+--------------+--------------------+
|payment_id|loan_id|payment_date|payment_amount| ingestion_timestamp|
+----------+-------+------------+--------------+--------------------+
|         1|     45|  2024-01-01|        100.00|2024-11-07 08:29:...|
|        10|     11|  2024-01-10|        550.00|2024-11-07 08:29:...|
|       100|     33|  2024-04-10|       1000.00|2024-11-07 08:29:...|
|        11|     22|  2024-01-11|        600.00|2024-11-07 08:29:...|
|        12|     33|  2024-01-12|        650.00|2024-11-07 08:29:...|
|        13|     44|  2024-01-13|        700.00|2024-11-07 08:29:...|
|        14|     55|  2024-01-14|        750.00|2024-11-07 08:29:...|
|        15|     66|  2024-01-15|        800.00|2024-11-07 08:29:...|
|        17|     88|  2024-01-17|        900.00|2024-11-07 08:29:...|
|        18|     99|  2024-01-18|        950.00|2024-11-07 08:29:...|
|        20|     21|  2024-01-20|       1050.00|2024-11-07 08:29:...|
|        21|     32|

In [0]:
df_loan_payments.columns

['payment_id',
 'loan_id',
 'payment_date',
 'payment_amount',
 'ingestion_timestamp']

In [0]:
df_loan_payments.printSchema()

root
 |-- payment_id: string (nullable = true)
 |-- loan_id: string (nullable = true)
 |-- payment_date: string (nullable = true)
 |-- payment_amount: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = false)



In [0]:
delta_customers_location = "delta/customers"
delta_accounts_location = "delta/accounts"
delta_transactions_location = "delta/transactions"
delta_loans_location = "delta/loans"
delta_loan_payments_location = "delta/loan_payments"

delta_customers_table_path = f"{mount_point_processed}/{delta_customers_location}"
delta_accounts_table_path = f"{mount_point_processed}/{delta_accounts_location}"
delta_transactions_table_path = f"{mount_point_processed}/{delta_transactions_location}"
delta_loans_table_path = f"{mount_point_processed}/{delta_loans_location}"
delta_loan_payments_table_path = f"{mount_point_processed}/{delta_loan_payments_location}"

In [0]:
from delta.tables import DeltaTable

if DeltaTable.isDeltaTable(spark, delta_customers_table_path):
    existing_data = DeltaTable.forPath(spark, delta_customers_table_path)
    
    (existing_data.alias("existing") \
        .merge(df_customers.alias("new"), "existing.customer_id = new.customer_id") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute())
else:
    df_customers.write.format("delta").mode("overwrite").save(delta_customers_table_path)

spark.read.format("delta").load(delta_customers_table_path).show(15)

+-----------+-----------+---------+---------------+-------------+-----+------+--------------------+
|customer_id| first_name|last_name|        address|         city|state|   zip| ingestion_timestamp|
+-----------+-----------+---------+---------------+-------------+-----+------+--------------------+
|          1|       John|      Doe|     123 Elm St|      Toronto|   ON|M4B1B3|2024-11-07 08:29:...|
|         10|        Ava| Anderson|909 Cypress Ave|  Quebec City|   QC|G1A0A1|2024-11-07 08:29:...|
|         11|  Alexander|   Thomas| 1010 Willow Rd|   St. John's|   NL|A1A0A1|2024-11-07 08:29:...|
|         12|   Isabella|      Lee| 1111 Poplar St|  Fredericton|   NB|E3B0A1|2024-11-07 08:29:...|
|         13|     Daniel|   Harris|  1212 Ash Blvd|Charlottetown|   PE|C1A0A1|2024-11-07 08:29:...|
|         14|     Sophia|    Young|  1313 Beech Dr|  Yellowknife|   NT|X1A0A1|2024-11-07 08:29:...|
|         15|    Matthew|     King|  1414 Cedar Ln|   Whitehorse|   YT|Y1A0A1|2024-11-07 08:29:...|


In [0]:
if DeltaTable.isDeltaTable(spark, delta_accounts_table_path):
    existing_data = DeltaTable.forPath(spark, delta_accounts_table_path)
    
    (existing_data.alias("existing") \
        .merge(df_accounts.alias("new"), "existing.account_id = new.account_id") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute())
else:
    df_accounts.write.format("delta").mode("overwrite").save(delta_accounts_table_path)

spark.read.format("delta").load(delta_accounts_table_path).show(15)

+----------+-----------+------------+-------+--------------------+
|account_id|customer_id|account_type|balance| ingestion_timestamp|
+----------+-----------+------------+-------+--------------------+
|         1|         45|     Savings|1000.50|2024-11-07 08:29:...|
|        11|          3|     Savings|1100.75|2024-11-07 08:29:...|
|        13|         29|     Savings|1300.25|2024-11-07 08:29:...|
|        15|         47|     Savings| 700.75|2024-11-07 08:29:...|
|        16|         18|    Checking|1400.00|2024-11-07 08:29:...|
|        18|          5|    Checking|1600.50|2024-11-07 08:29:...|
|         2|         12|    Checking|2500.75|2024-11-07 08:29:...|
|        20|         21|    Checking|2000.00|2024-11-07 08:29:...|
|        21|         53|     Savings| 300.25|2024-11-07 08:29:...|
|        22|         37|    Checking|2400.50|2024-11-07 08:29:...|
|        24|         11|    Checking|2600.00|2024-11-07 08:29:...|
|        26|         25|    Checking|2800.50|2024-11-07 08:29:

In [0]:
if DeltaTable.isDeltaTable(spark, delta_transactions_table_path):
    existing_data = DeltaTable.forPath(spark, delta_transactions_table_path)
    
    (existing_data.alias("existing") \
        .merge(df_transactions.alias("new"), "existing.transaction_id = new.transaction_id") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute())
else:
    df_transactions.write.format("delta").mode("overwrite").save(delta_transactions_table_path)

spark.read.format("delta").load(delta_transactions_table_path).show(15)

+--------------+----------+----------------+------------------+----------------+--------------------+
|transaction_id|account_id|transaction_date|transaction_amount|transaction_type| ingestion_timestamp|
+--------------+----------+----------------+------------------+----------------+--------------------+
|            13|        29|      2024-01-13|            150.00|         Deposit|2024-11-07 08:29:...|
|            14|        64|      2024-01-14|            300.25|      Withdrawal|2024-11-07 08:29:...|
|            16|        18|      2024-01-16|            175.00|      Withdrawal|2024-11-07 08:29:...|
|            18|         5|      2024-01-18|            275.75|      Withdrawal|2024-11-07 08:29:...|
|            20|        21|      2024-01-20|            375.25|      Withdrawal|2024-11-07 08:29:...|
|            24|        11|      2024-01-24|            300.25|      Withdrawal|2024-11-07 08:29:...|
|            25|        66|      2024-01-25|            250.00|         Deposit|20

In [0]:
if DeltaTable.isDeltaTable(spark, delta_loans_table_path):
    existing_data = DeltaTable.forPath(spark, delta_loans_table_path)
    
    (existing_data.alias("existing") \
        .merge(df_loans.alias("new"), "existing.loan_id = new.loan_id") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute())
else:
    df_loans.write.format("delta").mode("overwrite").save(delta_loans_table_path)

spark.read.format("delta").load(delta_loans_table_path).show(15)

+-------+-----------+-----------+-------------+---------+--------------------+
|loan_id|customer_id|loan_amount|interest_rate|loan_term| ingestion_timestamp|
+-------+-----------+-----------+-------------+---------+--------------------+
|      1|         45|   10000.50|          5.5|       36|2024-11-07 08:29:...|
|     11|          3|   10000.75|          6.0|       60|2024-11-07 08:29:...|
|     13|         29|   15000.25|          5.0|       36|2024-11-07 08:29:...|
|     15|         47|   25000.75|          6.5|       60|2024-11-07 08:29:...|
|     16|         18|   17500.00|          3.0|       24|2024-11-07 08:29:...|
|     18|          5|   27500.50|          4.5|       48|2024-11-07 08:29:...|
|      2|         12|   20000.75|          4.5|       48|2024-11-07 08:29:...|
|     20|         21|   37500.00|          3.5|       24|2024-11-07 08:29:...|
|     21|         53|   10000.25|          5.0|       36|2024-11-07 08:29:...|
|     22|         37|   20000.50|          4.0|     

In [0]:
if DeltaTable.isDeltaTable(spark, delta_loan_payments_table_path):
    existing_data = DeltaTable.forPath(spark, delta_loan_payments_table_path)
    
    (existing_data.alias("existing") \
        .merge(df_loan_payments.alias("new"), "existing.payment_id = new.payment_id") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute())
else:
    df_loan_payments.write.format("delta").mode("overwrite").save(delta_loan_payments_table_path)

spark.read.format("delta").load(delta_loan_payments_table_path).show(15)

+----------+-------+------------+--------------+--------------------+
|payment_id|loan_id|payment_date|payment_amount| ingestion_timestamp|
+----------+-------+------------+--------------+--------------------+
|        10|     11|  2024-01-10|        550.00|2024-11-07 08:29:...|
|        11|     22|  2024-01-11|        600.00|2024-11-07 08:29:...|
|        13|     44|  2024-01-13|        700.00|2024-11-07 08:29:...|
|        15|     66|  2024-01-15|        800.00|2024-11-07 08:29:...|
|        20|     21|  2024-01-20|       1050.00|2024-11-07 08:29:...|
|        21|     32|  2024-01-21|       1100.00|2024-11-07 08:29:...|
|        23|     54|  2024-01-23|       1200.00|2024-11-07 08:29:...|
|        28|      9|  2024-01-28|       1450.00|2024-11-07 08:29:...|
|        29|     20|  2024-01-29|       1500.00|2024-11-07 08:29:...|
|        31|     42|  2024-01-31|       1600.00|2024-11-07 08:29:...|
|        33|     64|  2024-02-02|       1700.00|2024-11-07 08:29:...|
|        39|     30|